# Add and norm

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_add_and_norm2.png" alt="Add and norm" style="width:425px;height:626px;">
</div>

Después del `Masked Multi-Head Attention` se pasa por una capa de `Add & Norm` que como ya hemos explicado da beneficios a la red por las conexiones residuales

## Implementación

Rescatamos la clase que ya hicimos

In [1]:
import torch
import torch.nn as nn

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))